In [1]:
import numpy as np
import glob, json
import argparse
import os, time, tqdm, datetime
import subprocess

parser = argparse.ArgumentParser()
parser.add_argument('--vid', type=int, nargs='+', default=[])
parser.add_argument('--local', action='store_true', default=False)
parser.add_argument('--mount_dir', required=True)
parser.add_argument('--dataset_dir', required=True)
parser.add_argument('--curr_vid', type=int, default=9)
parser.add_argument('--gen_missing_sj', action='store_true', default=False)

args = argparse.Namespace(
    vid = [8, 10],
    curr_vid = 9,
    mount_dir = '/data/mint/dataset_generation_symlink/',
    dataset_dir = '/data/mint/dataset_generation/',
    gen_missing_sj = False,
)

# Generate dataset path
name = 'random_target'
model_name = "/log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250/"
misc = "/ema_085000/train/render_face/reverse_sampling"
  
if __name__ == '__main__':
    
    t_start = time.time()
    sj_dict_v = {}
    sj_dict_all = {}
    for id in args.vid + [args.curr_vid]:
        sj_dict_v[id] = {}
        sj_folder = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/*')
        for sj in sj_folder:
            if 'filebrowser' in sj: continue
            sj_name = sj.split('/')[-1]
            target_name = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/{sj_name}/*')
            sj_dict_v[id][sj_name] = 1
            if sj_name not in sj_dict_all.keys():
                sj_dict_all[sj_name] = {'count': 1}
            else: 
                sj_dict_all[sj_name]['count'] += 1
        print(f"[#] V{id:02d} has {len(sj_dict_v[id])} sj...")
        print(f"[#] from path: {args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/*")
            
    print(f"[#] Total time: {time.time() - t_start}")
    print(f"[#] Total sj: {len(sj_dict_all.keys())}")
    
    template_sj = dict.fromkeys([f'src={i}.jpg' for i in range(0, 60000)])
    unvisited_sj = set(template_sj.keys()) - set(sj_dict_all.keys())
    print(f"[#] Unvisited sj: {unvisited_sj}")
    print(f"[#] Unvisited sj: {len(unvisited_sj)}")
    
    if args.gen_missing_sj:
        # Find index of unvisited sj in /home/mint/Dev/DiFaReli/difareli-faster/dataset_generation/sampler/generated_dataset_seed=47.json
        with open('/home/mint/Dev/DiFaReli/difareli-faster/dataset_generation/sampler/generated_dataset_seed=47.json', 'r') as f:
            dat = json.load(f)
            
        # Find pair-id based on unvisited_sj
        found_pairs = []
        found_id = []
        found_pairs_json = {'pair': {}}
        
        for sj in tqdm.tqdm(list(unvisited_sj)):
            for pair_id, pair_data in dat['pair'].items():
                if f'src={pair_data["src"]}' == sj:
                    found_pairs.append(pair_id)
                    found_id.append(pair_data["src"].split('=')[-1].split('.')[0])
                    found_pairs_json['pair'][pair_id] = pair_data
                    break

        # Print or use the found pair-ids as needed
        print("[#] Found pair-ids:", found_pairs)
        print("[#] Found pair-ids:", len(found_pairs))
        found_pairs_json['time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(f'/home/mint/Dev/DiFaReli/difareli-faster/sample_scripts/dataset_generation/sample_json/fill_sj_generated_dataset_seed=47.json', 'w') as f:
            json.dump(found_pairs_json, f, indent=4)
            
    # Just get the first pair of each sj to fulfill the coverage

[#] V08 has 36566 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v08/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*
[#] V10 has 38858 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v10/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*
[#] V09 has 60000 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v09/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*
[#] Total time: 1.440260648727417
[#] Total sj: 60000
[#] Unvisited sj: set()
[#] Unvisited sj: 0


In [2]:
import numpy as np
import glob, json
import argparse
import os, time, tqdm, datetime
import subprocess

parser = argparse.ArgumentParser()
parser.add_argument('--vid', type=int, nargs='+', default=[])
parser.add_argument('--local', action='store_true', default=False)
parser.add_argument('--mount_dir', required=True)
parser.add_argument('--dataset_dir', required=True)
parser.add_argument('--curr_vid', type=int, default=9)
parser.add_argument('--gen_missing_sj', action='store_true', default=False)

args = argparse.Namespace(
    vid = [8, 10],
    curr_vid = 9,
    mount_dir = '/data/mint/dataset_generation_symlink/',
    dataset_dir = '/data/mint/dataset_generation/',
    gen_missing_sj = False,
)

# Generate dataset path
name = 'random_target'
model_name = "/log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250/"
misc = "/ema_085000/train/render_face/reverse_sampling"
  
if __name__ == '__main__':
    
    t_start = time.time()
    sj_dict_v = {}
    sj_dict_all = {}
    count = 0
    generate_progress = {'pair': {}}
    
    
    for id in args.vid + [args.curr_vid]:
        sj_dict_v[id] = {}
        sj_folder = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/*')
        for sj in tqdm.tqdm(sj_folder):
            if 'filebrowser' in sj: continue
            src_sj_name = sj.split('/')[-1]
            target_name = glob.iglob(f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/{src_sj_name}/*')
            sj_dict_v[id][src_sj_name] = 1
            if src_sj_name not in sj_dict_all.keys():
                sj_dict_all[src_sj_name] = {'count': 1}
            else: 
                sj_dict_all[src_sj_name]['count'] += 1
            for tmp in target_name:
                dst_sj_name = tmp.split('/')[-1]
                generate_progress['pair'][f'pair{count}'] = {
                    "src": f"{src_sj_name.split('=')[-1]}",
                    "dst": f"{dst_sj_name.split('=')[-1]}",
                }
                count+=1
                
        print(f"[#] V{id:02d} has {len(sj_dict_v[id])} sj...")
        print(f"[#] from path: {args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/*")
            
    print(f"[#] Total time: {time.time() - t_start}")
    print(f"[#] Total sj: {len(sj_dict_all.keys())}")
    
    template_sj = dict.fromkeys([f'src={i}.jpg' for i in range(0, 60000)])
    unvisited_sj = set(template_sj.keys()) - set(sj_dict_all.keys())
    print(f"[#] Unvisited sj: {unvisited_sj}")
    print(f"[#] Unvisited sj: {len(unvisited_sj)}")
    
    generate_progress['time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(f'./sampling_progress_sj_generated_dataset.json', 'w') as f:
        json.dump(generate_progress, f, indent=4)
    

36566it [00:18, 1965.45it/s]


[#] V08 has 36566 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v08/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*


38859it [00:25, 1531.14it/s]


[#] V10 has 38858 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v10/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*


60000it [00:01, 32332.38it/s]


[#] V09 has 60000 sj...
[#] from path: /data/mint/dataset_generation_symlink//mount/v09/random_target//log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250///ema_085000/train/render_face/reverse_sampling/*
[#] Total time: 45.855520486831665
[#] Total sj: 60000
[#] Unvisited sj: set()
[#] Unvisited sj: 0


In [3]:
import numpy as np
import glob, json
import argparse
import os, time, tqdm, datetime
import subprocess

with open('./sampling_progress_sj_generated_dataset.json', 'r') as f:
    pairs = json.load(f)['pair']
    
sj_dict = {}
for k, v in pairs.items():
    if v['src'] not in sj_dict.keys():
        sj_dict[v['src']] = [v['dst']]
    else:
        sj_dict[v['src']].append(v['dst'])

max_len = {"n":0, "src_sj":None, "dst_sj":None}
min_len = {"n":100, "src_sj":None, "dst_sj":None}
sj_dict_f = {}
for k, v in sj_dict.items():
    # print("Before remove duplicate : ", k, v, len(v))
    # print("After remove duplicate : ", k, set(v), len(set(v)))
    sj_dict_f[k] = list(set(v))
    if len(set(v)) > max_len['n']:
        max_len['n'] = len(set(v))
        max_len['src_sj'] = k
        max_len['dst_sj'] = set(v)
    if len(set(v)) < min_len['n']:
        min_len['n'] = len(set(v))
        min_len['src_sj'] = k
        min_len['dst_sj'] = set(v)
        
    # print("="*50)

print("Max len : ", max_len)
print("Min len : ", min_len)

Max len :  {'n': 9, 'src_sj': '20586.jpg', 'dst_sj': {'56963.jpg', '4271.jpg', '27192.jpg', '58238.jpg', '38521.jpg', '25799.jpg', '24230.jpg', '36883.jpg', '31756.jpg'}}
Min len :  {'n': 1, 'src_sj': '21856.jpg', 'dst_sj': {'26058.jpg'}}


In [11]:
# Sampling more sj to match the max_len
import copy
import numpy as np
import pytorch_lightning as pl
seed = 47
pl.seed_everything(seed)
pool = [f'{i}.jpg' for i in range(0, 60000)]
n = max_len['n']
out = copy.deepcopy(sj_dict_f)
output_save = {}
for k, v in tqdm.tqdm(sj_dict_f.items()):
    if n <= len(sj_dict_f[k]):
        out[k] = v[:n]
    elif n > len(sj_dict_f[k]):
        to_add = n - len(sj_dict_f[k])
        available_sj = sorted(set(pool) - set(sj_dict_f[k]))
        tmp = np.random.choice(available_sj, to_add, replace=False)
        out[k] = v + list(tmp)
    else:
        print("Error")

Global seed set to 47
100%|██████████| 60000/60000 [24:33<00:00, 40.72it/s]


In [15]:
# Save file of the new sampling
import multiprocessing
print(k, out[k], sj_dict_f[k])


def process_set(data, index):
    out_save = {'pair': {}}
    count = 0
    num_sj = len(data.keys())
    for i in range(0, num_sj):
        src_sj = f'{i}.jpg'
        out_save['pair'][f'pair{count}'] = {
            'src': src_sj,
            'dst': data[src_sj][index]
        }
        count += 1
    with open(f'./sampling_generated_dataset/generated_dataset_{index+1}_seed={seed}.json', 'w') as f:
        json.dump(out_save, f, indent=4)
        
os.makedirs('./sampling_generated_dataset/', exist_ok=True)
with multiprocessing.Pool(processes=24) as pool:
        # Map process_chunk function to each chunk
        results = pool.starmap(process_set, zip([out]*n, range(0, n)))


22516.jpg ['6969.jpg', '19471.jpg', '44981.jpg', '26015.jpg', '35548.jpg', '7349.jpg', '954.jpg', '26867.jpg', '44528.jpg'] ['6969.jpg']


# Check progress of sampling

In [24]:
import numpy as np
import glob, json
import argparse
import os, time, tqdm, datetime
import subprocess

parser = argparse.ArgumentParser()
parser.add_argument('--vid', type=int, nargs='+', default=[])
parser.add_argument('--local', action='store_true', default=False)
parser.add_argument('--mount_dir', required=True)
parser.add_argument('--dataset_dir', required=True)
parser.add_argument('--curr_vid', type=int, default=9)
parser.add_argument('--gen_missing_sj', action='store_true', default=False)

args = argparse.Namespace(
    vid = [8, 10],
    curr_vid = 9,
    mount_dir = '/data/mint/dataset_generation_symlink/',
    dataset_dir = '/data/mint/dataset_generation/',
    gen_missing_sj = False,
)


def check_progress(idx):
    with open(f'./sampling_generated_dataset/generated_dataset_{idx}_seed=47.json') as f:
        data = json.load(f)['pair']
    name = 'random_target'
    model_name = "/log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml_step=250/"
    misc = "/ema_085000/train/render_face/reverse_sampling"
    postfix = 'Lerp_1000/n_frames=2/'
    
    found = []
    for k, v in tqdm.tqdm(data.items()):
        src_sj = v['src']
        dst_sj = v['dst']
        for id in args.vid + [args.curr_vid]:
            c_folder = f'{args.mount_dir}/mount/v{id:02d}/{name}/{model_name}/{misc}/src={src_sj}/dst={dst_sj}'
            c_file_recon = f'{c_folder}/{postfix}/res_frame0.png'
            c_file_relit = f'{c_folder}/{postfix}/res_frame1.png'
            if os.path.exists(c_file_recon) and os.path.exists(c_file_relit) and os.path.exists(c_folder):
                found.append(k)
                break
    return found

n_check = 3
with multiprocessing.Pool(processes=24) as pool:
        # Map process_chunk function to each chunk
        results = pool.starmap(check_progress, zip(range(0, n_check)))

  8%|▊         | 4767/60000 [00:33<06:58, 132.07it/s]